
### Image Composition with Pillow


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [ ]:
import os
from PIL import Image

def load_assets(asset_dir):
    assets = {}
    for file_name in os.listdir(asset_dir):
        if file_name.endswith(('png', 'jpg', 'jpeg')):
            key = os.path.splitext(file_name)[0]
            assets[key] = Image.open(os.path.join(asset_dir, file_name)).convert("RGBA")
    return assets

def compose_frame(base_image, assets, positions):
    composite_image = base_image.copy()
    for key, pos in positions.items():
        if key in assets:
            asset = assets[key]
            if 'resize' in pos:
                asset = asset.resize(pos['resize'])
            composite_image.paste(asset, pos['position'], asset)
    return composite_image

def save_frame(image, output_dir, frame_name):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    image.save(os.path.join(output_dir, f"{frame_name}.png"))

def create_ad_frames(asset_dirs, output_dir):
    for frame_type, asset_dir in asset_dirs.items():
        print(f"Processing {frame_type} frame...")
        assets = load_assets(asset_dir)
        base_image = assets.get('base_image')

        if not base_image:
            print(f"Base image not found for {frame_type} frame.")
            continue

        positions = {
            'preview': {
                'engagement_instruction': {'position': (50, 50), 'resize': (543, 84)},
                'logo': {'position': (50, 150), 'resize': (300, 50)},
                'product1': {'position': ((base_image.width - 400) // 2, (base_image.height - 400) // 2), 'resize': (400, 400)},
                'engagement_animation': {'position': ((base_image.width - 50) // 2, (base_image.height - 50) // 2), 'resize': (50, 50)},
                'product2': {'position': ((base_image.width - 400) // 2, 700), 'resize': (400, 200)},
            },
            'landing_frame': {
                'tagline2': {'position': (50, 680), 'resize': (300, 50)},
                'tagline1': {'position': (50, 50), 'resize': (543, 84)},
                'product1': {'position': ((base_image.width - 400) // 2, (base_image.height - 400) // 2), 'resize': (400, 400)},
                'engagement_animation': {'position': ((base_image.width - 50) // 2, (base_image.height - 50) // 2), 'resize': (50, 50)},
                'product2': {'position': (50, 750), 'resize': (500, 100)},
            },
            'endframe': {
                'tagline': {'position': (50, 50), 'resize': (543, 84)},
                'logo': {'position': (50, 750), 'resize': (300, 50)},
                'cta': {'position': (50, 800), 'resize': (150, 50)},
                'product1': {'position': ((base_image.width - 200) // 2, (base_image.height - 200) // 2), 'resize': (200, 200)},
                'ripple': {'position': ((base_image.width - 500) // 2, (base_image.height - 500) // 2), 'resize': (500, 500)},
                'engagement_animation': {'position': ((base_image.width - 50) // 2, (base_image.height - 50) // 2), 'resize': (50, 50)}
            }
        }

        frame_positions = positions.get(frame_type, {})
        composed_image = compose_frame(base_image, assets, frame_positions)
        save_frame(composed_image, output_dir, frame_type)

asset_dirs = {
    'preview': '/content/drive/MyDrive/wk12data/preview',
    'landing_frame': '/content/drive/MyDrive/wk12data/landing_frame',
    'endframe': '/content/drive/MyDrive/wk12data/endframe'
}
output_dir = '/content/drive/MyDrive/wk12data/output_frames'

create_ad_frames(asset_dirs, output_dir)

Processing preview frame...
Processing landing_frame frame...
Processing endframe frame...


### Image Composition with Autogen

In [25]:
! pip install pyautogen
! pip install openai
! pip install Pillow
! pip install datasets

In [27]:
import os
from autogen import AssistantAgent, UserProxyAgent
from PIL import Image
from openai import OpenAI

In [4]:
from google.colab import userdata

In [5]:
config_list = [
    {
        "model": "gpt-4",
        "api_key": userdata.get('OPENAI_API_KEY'),
    }
]

In [ ]:
def load_assets(asset_dir):
    assets = {}
    for file_name in os.listdir(asset_dir):
        if file_name.endswith(('png', 'jpg', 'jpeg')):
            key = os.path.splitext(file_name)[0]
            assets[key] = Image.open(os.path.join(asset_dir, file_name)).convert("RGBA")
    return assets

def compose_frame(base_image, assets, positions):
    composite_image = base_image.copy()
    for key, pos in positions.items():
        if key in assets:
            asset = assets[key]
            if 'resize' in pos:
                asset = asset.resize(pos['resize'])
            composite_image.paste(asset, pos['position'], asset)
    return composite_image

def save_frame(image, output_dir, frame_name):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    image.save(os.path.join(output_dir, f"{frame_name}.png"))

def create_ad_frames(asset_dirs, output_dir):
    for frame_type, asset_dir in asset_dirs.items():
        print(f"Processing {frame_type} ...")
        assets = load_assets(asset_dir)
        base_image = assets.get('base_image')

        if not base_image:
            print(f"Base image not found for {frame_type} frame.")
            continue

        positions = {
            'preview': {
                'engagement_instruction': {'position': (50, 50), 'resize': (543, 84)},
                'logo': {'position': (50, 150), 'resize': (300, 50)},
                'product1': {'position': ((base_image.width - 400) // 2, (base_image.height - 400) // 2), 'resize': (400, 400)},
                'engagement_animation': {'position': ((base_image.width - 50) // 2, (base_image.height - 50) // 2), 'resize': (50, 50)},
                'product2': {'position': ((base_image.width - 400) // 2, 700), 'resize': (400, 200)},
            },
            'landing_frame': {
                'tagline2': {'position': (50, 680), 'resize': (300, 50)},
                'tagline1': {'position': (50, 50), 'resize': (543, 84)},
                'product1': {'position': ((base_image.width - 400) // 2, (base_image.height - 400) // 2), 'resize': (400, 400)},
                'engagement_animation': {'position': ((base_image.width - 50) // 2, (base_image.height - 50) // 2), 'resize': (50, 50)},
                'product2': {'position': (50, 750), 'resize': (500, 100)},
            },
            'endframe': {
                'tagline': {'position': (50, 50), 'resize': (543, 84)},
                'logo': {'position': (50, 750), 'resize': (300, 50)},
                'cta': {'position': (50, 800), 'resize': (150, 50)},
                'product1': {'position': ((base_image.width - 200) // 2, (base_image.height - 200) // 2), 'resize': (200, 200)},
                'ripple': {'position': ((base_image.width - 500) // 2, (base_image.height - 500) // 2), 'resize': (500, 500)},
                'engagement_animation': {'position': ((base_image.width - 50) // 2, (base_image.height - 50) // 2), 'resize': (50, 50)}
            }
        }

        frame_positions = positions.get(frame_type, {})
        composed_image = compose_frame(base_image, assets, frame_positions)
        save_frame(composed_image, output_dir, frame_type)

asset_dirs = {
    "preview": "/content/drive/MyDrive/wk12data/preview",
    "landing_frame": "/content/drive/MyDrive/wk12data/landing_frame",
    "endframe": "/content/drive/MyDrive/wk12data/endframe"
}
output_dir = "/content/drive/MyDrive/wk12data/output"

llm_config = {
    "functions": [
        {
            "name": "create_ad_frames",
            "description": "Compose an ad frame using the given assets",
            "parameters": {
                "type": "object",
                "properties": {
                    "asset_dirs": {
                        "type": "object",
                        "description": "Dictionary of asset directories for different frame types",
                    },
                    "output_dir": {
                        "type": "string",
                        "description": "Directory to save the composed frames",
                    },
                },
                "required": ["asset_dirs", "output_dir"],
            },
        },
    ],
    "config_list": config_list,
    "timeout": 120,
}

chatbot = AssistantAgent(
    name="chatbot",
    system_message="For coding tasks, only use the functions you have been provided with. Reply TERMINATE when the task is done.",
    llm_config=llm_config,
)

user_proxy = UserProxyAgent(
    name="user_proxy",
    #is_termination_msg=lambda x: x and x.get("content", "").rstrip().endswith("TERMINATE"),
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={
        "use_docker": False,
    },
)

user_proxy.register_function(
    function_map={
        "create_ad_frames": create_ad_frames,
    }
)

user_proxy.initiate_chat(
    chatbot,
    message=f"Create ad frames using the provided asset directories {asset_dirs} and save the output to {output_dir}. TERMINATE",
)


user_proxy (to chatbot):

Create ad frames using the provided asset directories {'preview': '/content/drive/MyDrive/wk12data/preview', 'landing_frame': '/content/drive/MyDrive/wk12data/landing_frame', 'endframe': '/content/drive/MyDrive/wk12data/endframe'} and save the output to /content/drive/MyDrive/wk12data/output. TERMINATE

--------------------------------------------------------------------------------
chatbot (to user_proxy):

***** Suggested function call: create_ad_frames *****
Arguments: 
{
"output_dir": "/content/drive/MyDrive/wk12data/output"
}
*****************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION create_ad_frames...
user_proxy (to chatbot):

***** Response from calling function (create_ad_frames) *****
Error: create_ad_frames() missing 1 required positional argument: 'asset_dirs'
*************************************************************

----------------------

ChatResult(chat_id=None, chat_history=[{'content': "Create ad frames using the provided asset directories {'preview': '/content/drive/MyDrive/wk12data/preview', 'landing_frame': '/content/drive/MyDrive/wk12data/landing_frame', 'endframe': '/content/drive/MyDrive/wk12data/endframe'} and save the output to /content/drive/MyDrive/wk12data/output. TERMINATE", 'role': 'assistant'}, {'function_call': {'arguments': '{\n"output_dir": "/content/drive/MyDrive/wk12data/output"\n}', 'name': 'create_ad_frames'}, 'content': None, 'role': 'assistant'}, {'content': "Error: create_ad_frames() missing 1 required positional argument: 'asset_dirs'", 'name': 'create_ad_frames', 'role': 'function'}, {'content': "It seems like I made a mistake. I missed out the required argument 'asset_dirs'. Let me correct that.", 'function_call': {'arguments': '{\n"output_dir": "/content/drive/MyDrive/wk12data/output",\n"asset_dirs": {\'preview\': \'/content/drive/MyDrive/wk12data/preview\', \'landing_frame\': \'/content/d

### GPT-4o Vision for image review

In [31]:
import base64
import requests

# OpenAI API Key
api_key = userdata.get('OPENAI_API_KEY')

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

image_path = "/content/drive/MyDrive/wk12data/output/endframe.png"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {api_key}"
}

payload = {
  "model": "gpt-4o",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Please provide a description of the image and then rate, on a scale of 1 to 10, how closely the image satisfies design principles of advertisement"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-9lG3MaJ1TkIAXkhxSt0TvZuHIZtCI', 'object': 'chat.completion', 'created': 1721050152, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '### Description\n\nThe advertisement features a gradient blue background transitioning from light blue to dark blue. At the top, white text reads, "Maximize return potential Tap into sound opportunities." Below this, there is an icon resembling a mobile phone\'s status bar displaying the time (18:01), along with battery, Wi-Fi, and signal icons.\n\nCentral to the design is a circular graphic with concentric circles emanating from the center. Inside the circle, a blue background features a red triangle play button labeled "ASIA," surrounded by illustrated high-rise buildings. This imagery likely represents the Asian financial market. Below the circular graphic, yellow text reads, "Schroders Asian Asset Income Strategy." Finally, a blue "Learn More" button, accented with a graphic symbol, a

{'id': 'chatcmpl-9lG3MaJ1TkIAXkhxSt0TvZuHIZtCI', 'object': 'chat.completion', 'created': 1721050152, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '### Description\n\nThe advertisement features a gradient blue background transitioning from light blue to dark blue. At the top, white text reads, "Maximize return potential Tap into sound opportunities." Below this, there is an icon resembling a mobile phone\'s status bar displaying the time (18:01), along with battery, Wi-Fi, and signal icons.\n\nCentral to the design is a circular graphic with concentric circles emanating from the center. Inside the circle, a blue background features a red triangle play button labeled "ASIA," surrounded by illustrated high-rise buildings. This imagery likely represents the Asian financial market. Below the circular graphic, yellow text reads, "Schroders Asian Asset Income Strategy." Finally, a blue "Learn More" button, accented with a graphic symbol, appears near the bottom.\n\n### Rating on Design Principles of Advertisement\n\n1. **Clarity of Message (8/10):** The message is clear and concise. The primary message about maximizing returns and sound opportunities is prominently displayed at the top, making it immediately noticeable.\n\n2. **Visual Hierarchy (7/10):** The use of size, color, and positioning effectively directs the viewer\'s attention from the top message to the central graphic, and then to the call-to-action at the bottom. However, the graphic\'s details might distract slightly from the main message.\n\n3. **Aesthetic Appeal (8/10):** The design uses harmonious colors and modern graphics, which are visually appealing. The gradient background and'}, 'logprobs': None, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 802, 'completion_tokens': 300, 'total_tokens': 1102}, 'system_fingerprint': 'fp_298125635f'}
